In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [8]:
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
# dispatch.dispatch_experiments('feb22', base_job_name='infocompare', game_name='large_game_2', overrides = overrides, n_seeds=10)
dispatch.dispatch_experiments('feb22', base_job_name='infocompare', game_name='large_game_2_hide', overrides = overrides, n_seeds=10)

Submitted batch job 3983
Submitted batch job 3984
Submitted batch job 3985
Submitted batch job 3986
Submitted batch job 3987
Submitted batch job 3988
Submitted batch job 3989
Submitted batch job 3990
Submitted batch job 3991
Submitted batch job 3992


In [19]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "_mlp  R" | awk '{print $1}' | xargs scancel

In [12]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)
              3984                        large_game_2_hide-feb22mlp-101_infocompare  R   19:34:34      1 ip-10-255-7-17
              3985                        large_game_2_hide-feb22mlp-102_infocompare  R   19:34:34      1 ip-10-255-7-63
              3986                        large_game_2_hide-feb22mlp-103_infocompare  R   19:34:34      1 ip-10-255-7-80
              3987                        large_game_2_hide-feb22mlp-104_infocompare  R   19:34:34      1 ip-10-255-7-81
              3988                        large_game_2_hide-feb22mlp-105_infocompare  R   19:34:34      1 ip-10-255-7-96
              3989                        large_game_2_hide-feb22mlp-106_infocompare  R   19:34:34      1 ip-10-255-7-101
              3990                        large_game_2_hide-feb22mlp-107_infocompare  R   19:34:34      1 ip-10-255-7-106
              3991          

In [79]:
# !scancel 5306 5307

In [ ]:
!scancel -u ubuntu

In [112]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

Submitted batch job 2422
Submitted batch job 2423
